In [16]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("하이닉스_임시.csv", thousands=',', encoding = 'cp949')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else -1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Calculating correlations for Forward Selection
correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
correlations.columns = ['Feature', 'Correlation']

#Performing T-tests
t_tests = {column: ttest_ind(X[column][y == 1], X[column][y == -1], nan_policy='omit') for column in X.columns}
t_tests_sorted = sorted(t_tests.items(), key=lambda x: x[1].pvalue)
t_tests_df = pd.DataFrame(t_tests_sorted, columns=['Feature', 'T-test'])
t_tests_df['T-test'] = t_tests_df['T-test'].apply(lambda x: x.pvalue)  # Only keep p-value for simplicity

# Lasso and Ridge regression
lasso = LassoCV().fit(X, y)
ridge = RidgeCV().fit(X, y)

lasso_importance = np.abs(lasso.coef_)
ridge_importance = np.abs(ridge.coef_)

features_lasso_sorted = sorted(zip(X.columns, lasso_importance), key=lambda x: x[1], reverse=True)
features_ridge_sorted = sorted(zip(X.columns, ridge_importance), key=lambda x: x[1], reverse=True)

features_lasso_df = pd.DataFrame(features_lasso_sorted, columns=['Feature', 'Lasso Importance'])
features_ridge_df = pd.DataFrame(features_ridge_sorted, columns=['Feature', 'Ridge Importance'])

# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# Combining all the data into a single dataframe without merging by feature
final_df = pd.concat([correlations, t_tests_df.drop('Feature', axis=1), features_lasso_df.drop('Feature', axis=1), 
                      features_ridge_df.drop('Feature', axis=1), features_rf_df.drop('Feature', axis=1)], axis=1)

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_8_features = features_rf_sorted[:8]

In [20]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("하이닉스_임시.csv", thousands=',', encoding = 'cp949')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else -1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Calculating correlations for Forward Selection
correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
correlations.columns = ['Feature', 'Correlation']

#Performing T-tests
t_tests = {column: ttest_ind(X[column][y == 1], X[column][y == -1], nan_policy='omit') for column in X.columns}
t_tests_sorted = sorted(t_tests.items(), key=lambda x: x[1].pvalue)
t_tests_df = pd.DataFrame(t_tests_sorted, columns=['Feature', 'T-test'])
t_tests_df['T-test'] = t_tests_df['T-test'].apply(lambda x: x.pvalue)  # Only keep p-value for simplicity

# Lasso and Ridge regression
lasso = LassoCV().fit(X, y)
ridge = RidgeCV().fit(X, y)

lasso_importance = np.abs(lasso.coef_)
ridge_importance = np.abs(ridge.coef_)

features_lasso_sorted = sorted(zip(X.columns, lasso_importance), key=lambda x: x[1], reverse=True)
features_ridge_sorted = sorted(zip(X.columns, ridge_importance), key=lambda x: x[1], reverse=True)

features_lasso_df = pd.DataFrame(features_lasso_sorted, columns=['Feature', 'Lasso Importance'])
features_ridge_df = pd.DataFrame(features_ridge_sorted, columns=['Feature', 'Ridge Importance'])

# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# Combining all the data into a single dataframe without merging by feature
final_df = pd.concat([correlations, t_tests_df.drop('Feature', axis=1), features_lasso_df.drop('Feature', axis=1), 
                      features_ridge_df.drop('Feature', axis=1), features_rf_df.drop('Feature', axis=1)], axis=1)

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_8_features = [feature for feature, importance in features_rf_sorted[:8]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_8_features]
y = df['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=42)

# 모델 초기화
models = {
    'SVC': SVC(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier()
}

# 점수를 저장할 딕셔너리
scores = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1_Score': []
}

# 각 모델에 대해 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    scores['Model'].append(model_name)
    scores['Accuracy'].append(accuracy_score(y_test, y_pred))
    scores['Precision'].append(precision_score(y_test, y_pred, average='macro'))
    scores['Recall'].append(recall_score(y_test, y_pred, average='macro'))
    scores['F1_Score'].append(f1_score(y_test, y_pred, average='macro'))

# 점수 딕셔너리를 데이터프레임으로 변환
scores_df = pd.DataFrame(scores)
scores_df

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Accuracy,Precision,Recall,F1_Score
0,SVC,0.520000,0.260000,0.500000,0.342105
1,LogisticRegression,0.506667,0.256757,0.487179,0.336283
2,DecisionTreeClassifier,0.480000,0.475806,0.476496,0.474015
3,RandomForestClassifier,0.520000,0.516667,0.516026,0.512987


In [7]:
features_rf_df

,Feature,RF Importance
0,시가,0.419807
1,종가,0.402909
2,MACD_Binary,0.035492
3,이평 비교,0.032089
4,RSI50이상,0.028401
5,MACD비교,0.021848
6,MACD_minus_Signal_Binary,0.021246
7,RSI비교,0.019230
8,RSI_minus_Signal_Binary,0.018978
